# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

In [12]:
import json
import requests
import pandas as pd
from pandas import json_normalize
from urllib.request import urlopen

In [17]:
url = 'https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset=100'

response = urlopen(url)
results = json.load(response)

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

In [20]:
flattened_data = json_normalize(results)
flattened_data1 = json_normalize(flattened_data.articles[0])
flattened_data1.head(5)

,sku,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,outfits,amount
0,PI922O0K7-G11,T-shirt imprimé - bordeaux,"[XXS, XS, S, M, L, XL]",pier-one-t-shirt-imprime-bordeaux-pi922o0k7-g11,[{'path': 'spp-media-p1/23f7bb5188393048b070f8...,Pier One,False,[],"[{'key': 'discountRate', 'value': '-10%', 'tra...",clothing,[],"12,99 €","11,69 €",False,False,False,False,NaN,NaN
1,JA222G0W2-K11,JJIGLENN JJORIGINAL - Jean slim - blue denim,"[27x30, 27x32, 28x30, 28x32, 29x30, 29x32, 30x...",jack-and-jones-jjiglenn-jjoriginal-jean-slim-b...,[{'path': 'spp-media-p1/47ce66518c9c3107999dce...,Jack & Jones,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -25...",clothing,[],"39,99 €","29,99 €",True,False,True,False,NaN,NaN
2,LE212O00O-A11,DECLAN MILLSTONE - Baskets basses - regular white,"[40, 41, 42, 43, 44, 45, 46]",levisr-declan-millstone-baskets-basses-regular...,[{'path': 'spp-media-p1/9b0da380bc203efa9907b8...,Levi's®,False,[],"[{'key': 'discountRate', 'value': '-25%', 'tra...",shoe,[],"49,95 €","37,65 €",False,False,False,False,NaN,NaN
3,TO722O0NS-M11,STRIPED PANELPRINT - T-shirt imprimé - woodlan...,"[XS, S, M, L, XL, XXL]",tom-tailor-denim-striped-panel-t-shirt-imprime...,[{'path': 'spp-media-p1/da50576cee9732ecad10ed...,TOM TAILOR DENIM,False,[],"[{'key': 'discountRate', 'value': '-35%', 'tra...",clothing,[],"15,99 €","10,44 €",False,False,False,False,NaN,NaN
4,NE212O055-A11,500 - Baskets basses - white,"[40, 40.5, 41.5, 42, 42.5, 43, 44, 44.5, 45, 4...",new-balance-500-baskets-basses-white-ne212o055...,[{'path': 'spp-media-p1/c4bc00d728303550b2de6b...,New Balance,False,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",shoe,[],"69,95 €","48,95 €",False,False,False,False,NaN,NaN


In [26]:
total_pages=results['pagination']['page_count']

df=pd.DataFrame()
for i in range(total_pages):
    k=84*i
    url=f'https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset={k}&sort=sale'
    response = urlopen(url)
    results = json.load(response)
    flattened_data = json_normalize(results)
    flattened_data1 = json_normalize(flattened_data.articles[0])
    flattened_data1=flattened_data1.set_index('sku')
    df = df.append(flattened_data1)

df.head(10)

,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,...,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,outfits,amount,condition,condition_key,price.base_price
sku,,,,,,,,,,,,,,,,,,,,,
HY554B01L-Q11,Masque en tissu - black,[One Size],hype-masque-en-tissu-black-hy554b01l-q11,[{'path': 'spp-media-p1/c15457b8f18532a086b55a...,Hype,False,[],"[{'key': 'discountRate', 'value': '-70%', 'tra...",accessoires,[],...,"4,50 €",False,False,False,False,NaN,NaN,NaN,NaN,NaN
TO122O08I-Q11,LOGO TEE - T-shirt imprimé - black,"[XS, S, M, L, XL, XXL, 3XL]",tommy-hilfiger-logo-tee-t-shirt-imprime-to122o...,[{'path': 'spp-media-p1/6a1582f6fde836a5917156...,Tommy Hilfiger,False,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,[],...,"27,95 €",False,False,False,False,NaN,NaN,NaN,NaN,NaN
JA222T0B6-O12,JJEROCKY JACKET - Veste en similicuir - cognac,"[XS, S, M, L, XL]",jack-and-jones-jjerocky-jacket-veste-en-simili...,[{'path': 'spp-media-p1/ae896c064f6d3147872443...,Jack & Jones,False,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,[],...,"41,99 €",False,False,False,False,NaN,NaN,NaN,NaN,NaN
OS322T05O-Q11,ONSMIKE RACER - Veste en similicuir - black,"[XS, S, M, L, XL, XXL]",only-and-sons-onsmike-racer-jacket-veste-en-si...,[{'path': 'spp-media-p1/5a76970b0d383d5d9dbf64...,Only & Sons,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -50...",clothing,[],...,"19,99 €",True,False,True,False,NaN,NaN,NaN,NaN,NaN
JA222G0W2-K11,JJIGLENN JJORIGINAL - Jean slim - blue denim,"[27x30, 27x32, 28x30, 28x32, 29x30, 29x32, 30x...",jack-and-jones-jjiglenn-jjoriginal-jean-slim-b...,[{'path': 'spp-media-p1/47ce66518c9c3107999dce...,Jack & Jones,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -25...",clothing,[],...,"29,99 €",True,False,True,False,NaN,NaN,NaN,NaN,NaN
JA222E0F5-K11,MARCO BOWIE - Chino - navy,"[27x30, 27x32, 28x30, 28x32, 29x30, 29x32, 29x...",jack-and-jones-marco-bowie-chino-navy-ja222e0f...,[{'path': 'spp-media-p1/33fc78cacde137fc8cecd1...,Jack & Jones,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -25...",clothing,[],...,"29,99 €",True,False,True,False,NaN,NaN,NaN,NaN,NaN
H0422O02T-A11,CREW CHAIN 3 PACK - T-shirt basique - white/gr...,"[XS, S, M, L, XL, XXL]",hollister-co-crew-3-pack-t-shirt-basique-white...,[{'path': 'spp-media-p1/dcc3647921c532de99f120...,Hollister Co.,False,[],"[{'key': 'discountRate', 'value': '-15%', 'tra...",clothing,[],...,"29,85 €",False,False,False,False,NaN,NaN,NaN,NaN,NaN
JA222E0FF-Q11,MARCO BOWIE - Chino - black,"[27x30, 27x32, 28x30, 28x32, 29x30, 29x32, 29x...",jack-and-jones-marco-bowie-chino-black-ja222e0...,[{'path': 'spp-media-p1/d3a38e839953316f8f2492...,Jack & Jones,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -30...",clothing,[],...,"27,99 €",True,False,True,False,NaN,NaN,NaN,NaN,NaN
UR622E01B-Q11,Pantalon cargo - black,"[XS, S, M, L, XL, XXL]",urban-classics-jogging-pant-pantalon-cargo-ur6...,[{'path': 'spp-media-p1/610c43f35ca7328b83954a...,Urban Classics,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -15...",clothing,[],...,"42,49 €",True,False,True,False,NaN,NaN,NaN,NaN,NaN


## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).